In [7]:
import pandas as pd

data = pd.read_csv('../preprocess/202111171547_all.csv')
df = pd.DataFrame(data)
df = df.dropna()
data = df.iloc[:, :-1]
target = df['target']

In [8]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, stratify=target, random_state=0)
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.2, stratify=train_target, random_state=0)

In [9]:
from tensorflow import keras
import tensorflow.keras.backend as K

K.clear_session()
model = keras.Sequential()
model.add(keras.layers.GRU(15, activation="tanh", input_shape=(train_input.shape[1],1), return_sequences=True))
model.add(keras.layers.GRU(7, activation="tanh", dropout=0.3))
model.add(keras.layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 160, 15)           810       
                                                                 
 gru_1 (GRU)                 (None, 7)                 504       
                                                                 
 dense (Dense)               (None, 1)                 8         
                                                                 
Total params: 1,322
Trainable params: 1,322
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint('11241615-best-gru-model.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 15, restore_best_weights=True)

history = model.fit(train_input, train_target, 
                    epochs=100, batch_size = 20,
                    validation_data = (val_input, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
159/159 [==============================] - ETA: 0s - loss: 0.5433 - accuracy: 0.7483

In [19]:
model.evaluate(val_input, val_target)

25/25 [==============================] - 0s 14ms/step - loss: 0.3092 - accuracy: 0.8717


[0.30923259258270264, 0.8716981410980225]

In [20]:
model.evaluate(test_input, test_target)

32/32 [==============================] - 0s 13ms/step - loss: 0.3326 - accuracy: 0.8934


[0.33256348967552185, 0.8933601379394531]